In [ ]:
import os
from datetime import UTC, datetime

from rich import print as rprint

from entitysdk import Client, ProjectContext, models

entitycore_api_url = "http://127.0.0.1:8000"
project_context = ProjectContext(
    virtual_lab_id="a98b7abc-fc46-4700-9e3d-37137812c730",
    project_id="0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6",
)
token = os.getenv("ACCESS_TOKEN", "XXX")
client = Client(api_url=entitycore_api_url, project_context=project_context, token_manager=token)

## Register a Circuit

In [ ]:
species = client.search_entity(
    entity_type=models.Species, query={"name__ilike": "Homo sapiens"}
).one()

subject = models.Subject(
    name="my-subject", description="my-subject-description", sex="male", species=species
)
subject = client.register_entity(subject)

brain_region = client.search_entity(entity_type=models.BrainRegion, query={"acronym": "CB"}).first()


circuit = models.Circuit(
    name="my-circuit",
    description="my-circuit",
    subject=subject,
    brain_region=brain_region,
    number_synapses=2,
    number_neurons=5,
    number_connections=10,
    scale="microcircuit",
    build_category="em_reconstruction",
)

circuit = client.register_entity(circuit)

# 1. Register the entire provenance at once
When all entities are available up front the entire provenance can be generated using the creation endpoints without any updating taking place.

## Register all entities

In [ ]:
campaign = client.register_entity(
    models.SimulationCampaign(
        name="my-campaign",
        description="my-campaign-description",
        entity_id=circuit.id,
        scan_parameters={"foo": "bar"},
    )
)

simulations = [
    client.register_entity(
        models.Simulation(
            name=f"sim-{i}",
            description=f"sim-{i}",
            scan_parameters={"foo": "bar"},
            entity_id=circuit.id,
            simulation_campaign_id=campaign.id,
        )
    )
    for i in range(5)
]
simulation_results = [
    client.register_entity(
        models.SimulationResult(
            name=f"result-{i}",
            description=f"result-{i}",
            simulation_id=simulations[i].id,
        )
    )
    for i in range(5)
]

## Register activities

In [ ]:
simulation_generation = client.register_entity(
    models.SimulationGeneration(
        start_time=datetime.now(UTC),
        used=[campaign],
        generated=simulations,
    )
)

simulation_executions = [
    client.register_entity(
        models.SimulationExecution(
            used=[simulations[i]],
            generated=[simulation_results[i]],
            start_time=datetime.now(UTC),
            status="done",
        )
    )
    for i in range(5)
]

In [ ]:
rprint(simulation_generation)
rprint(simulation_executions)

# 2. Register provenance gradually

## Register a SimulationCampaign

In [ ]:
campaign = client.register_entity(
    models.SimulationCampaign(
        name="my-campaign",
        description="my-campaign-description",
        entity_id=circuit.id,
        scan_parameters={"foo": "bar"},
    )
)

## Register SimulationGeneration

In [ ]:
sim_generation = client.register_entity(
    models.SimulationGeneration(
        start_time=datetime.now(UTC),
        used=[campaign],
    )
)

## Register Simulations

In [ ]:
simulations = [
    client.register_entity(
        models.Simulation(
            name=f"sim-{i}",
            description=f"sim-{i}",
            scan_parameters={"foo": "bar"},
            entity_id=circuit.id,
            simulation_campaign_id=campaign.id,
        )
    )
    for i in range(5)
]

## Update SimulationGeneration with simulations

In [ ]:
data_to_update = {
    "generated_ids": [s.id for s in simulations],
    "end_time": datetime.now(UTC),
}

updated_sim_generation = client.update_entity(
    entity_id=sim_generation.id, entity_type=type(sim_generation), attrs_or_entity=data_to_update
)

In [ ]:
rprint(updated_sim_generation)

## Create simulation executions

In [ ]:
executions = [
    client.register_entity(
        models.SimulationExecution(
            used=[simulations[i]],
            start_time=datetime.now(UTC),
            status="created",
        )
    )
    for i in range(5)
]

## Create simulation results

In [ ]:
results = [
    client.register_entity(
        models.SimulationResult(
            name=f"result-{i}",
            description=f"result-{i}",
            simulation_id=simulations[i].id,
        )
    )
    for i in range(5)
]

## Update simulation executions with results

In [ ]:
updated_executions = [
    client.update_entity(
        entity_id=executions[i].id,
        entity_type=models.SimulationExecution,
        attrs_or_entity={
            "generated_ids": [results[i].id],
            "end_time": datetime.now(UTC),
            "status": "done",
        },
    )
    for i in range(5)
]

In [ ]:
rprint(updated_executions)